In [2]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.models import Sequential
#from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import GlobalMaxPool1D,Embedding,Bidirectional,LSTM
import jieba
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('sohu.csv')

data.head()

,label,text
0,pic,高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑...
1,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
2,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
3,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
4,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...


In [3]:
data['label'].unique()

array(['pic', 'news', 'sports', 'business', 'caipiao', 'yule', 'mil',
       'cul'], dtype=object)

In [4]:
data = data[data['label'] != 'mil']
data = data[data['label'] != 'pic']
data = data[data['label'] != 'business']
#data = data[data['label'] != 'caipiao']
data = data[data['label'] != 'yule']
data = data[data['label'] != 'cul']

In [5]:
data['label'] = data['label'].map({'news': 0, 'sports': 1, 'caipiao': 2})

In [6]:
for i in data['text']:
    i = jieba.cut(i)

In [7]:
data['text'] = data['text'].apply(lambda x:' '.join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\CZY\AppData\Local\Temp\jieba.cache
Loading model cost 0.743 seconds.
Prefix dict has been built succesfully.


In [8]:
max_features = 10000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['text'])
list_tokenized_train = tokenizer.texts_to_sequences(data['text'])

from keras.preprocessing.sequence import pad_sequences
len = 100
x = pad_sequences(list_tokenized_train, maxlen = len)
y = data['label']

In [10]:
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x, y, test_size=0.3)

In [11]:
y1_train_onehot = np_utils.to_categorical(y1_train)   #转为二元矩阵
y1_test_onehot = np_utils.to_categorical(y1_test)

In [14]:
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(3,activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
batch_size = 50
epochs = 6
model.fit(x1_train, y1_train_onehot, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 2370 samples, validate on 593 samples
Epoch 1/6
2370/2370 [==============================] - 7s 3ms/step - loss: 0.5358 - acc: 0.7931 - val_loss: 0.3726 - val_acc: 0.8066
Epoch 2/6
2370/2370 [==============================] - 5s 2ms/step - loss: 0.2426 - acc: 0.8987 - val_loss: 0.1495 - val_acc: 0.9736
Epoch 3/6
2370/2370 [==============================] - 5s 2ms/step - loss: 0.0906 - acc: 0.9802 - val_loss: 0.0874 - val_acc: 0.9741
Epoch 4/6
2370/2370 [==============================] - 4s 2ms/step - loss: 0.0460 - acc: 0.9878 - val_loss: 0.0721 - val_acc: 0.9803
Epoch 5/6
2370/2370 [==============================] - 4s 2ms/step - loss: 0.0309 - acc: 0.9890 - val_loss: 0.0536 - val_acc: 0.9820
Epoch 6/6
2370/2370 [==============================] - 4s 2ms/step - loss: 0.0188 - acc: 0.9934 - val_loss: 0.0568 - val_acc: 0.9826


In [16]:
accuracy = model.evaluate(x1_test, y1_test_onehot, batch_size = 50)
print("test accuracy:{}".format(accuracy[1]))

1271/1271 [==============================] - 0s 305us/step
test accuracy:0.9766588117267839
